<a href="https://colab.research.google.com/github/soumitrapy/da6401_assignment2/blob/main/partB/notebook2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !git clone https://github.com/soumitrapy/da6401_assignment2.git project
# %cd project
# !wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
# !unzip -q nature_12K.zip -d data/

Cloning into 'project'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 33 (delta 8), reused 28 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 16.90 KiB | 692.00 KiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/project
--2025-04-22 04:52:19--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.207, 172.217.194.207, 142.250.4.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  20.8MB/s    in 2m 52s  

2025-04-22 04:55:11 (21.2 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [2]:
# !git pull
# %cd partB

/content/project/partB


In [3]:
import wandb
import yaml
import os
from datetime import datetime

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset,DataLoader, random_split

from torchvision import transforms
#from torchvision.io import read_image

In [4]:
import yaml
config = yaml.safe_load(open("config/pretrained.yaml"))

# config['dataset']['batch_size']=32
config['train']['epochs']=20
config


{'project': 'dla2partb',
 'use_wandb': True,
 'dataset': {'name': 'CustomDataset',
  'path': '../data/inaturalist_12K/',
  'img_size': 224,
  'class_names': ['Plantae',
   'Mammalia',
   'Animalia',
   'Reptilia',
   'Amphibia',
   'Aves',
   'Fungi',
   'Arachnida',
   'Mollusca',
   'Insecta'],
  'batch_size': 100},
 'model': {'name': 'resnet50', 'in_channels': 3, 'num_classes': 10},
 'train': {'epochs': 20, 'val_interval': 1}}

In [5]:
import wandb
if config.get('use_wandb',False):
    wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: soumitrapy (soumitrapy-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


### DataLoader Creation

In [6]:
from preprocessing import CustomDataset
from torchvision import transforms
import torch

# Data augmentation and normalization for training
# Just normalization for validation
cfg = config['dataset']

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(cfg['img_size']),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.RandomResizedCrop(cfg['img_size']),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
splits = ['train', 'val']
datasets = {x:CustomDataset(path= cfg['path']+x,
                            class_names=cfg['class_names'],
                            transform=data_transforms[x])
            for x in splits}

dataloaders = {x: torch.utils.data.DataLoader(datasets[x],
                                              batch_size=cfg['batch_size'],
                                              shuffle=True
                                              )
                for x in splits}

class_names = datasets['train'].class_names
print(f"train len: {len(datasets['train'])}, val len: {len(datasets['val'])}")

# We want to be able to train our model on an `accelerator <https://pytorch.org/docs/stable/torch.html#accelerators>`__
# such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"device = {device}")

train len: 9999, val len: 2000
device = cuda


In [7]:
for x, y in dataloaders['train']:
    print(x.shape,y.shape)
    break

torch.Size([100, 3, 224, 224]) torch.Size([100])


### Pretrained Model

In [8]:
from models import get_model
model = get_model(config['model'])
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True
model

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 65.6MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.01)

### Training Loop

In [10]:
from train import train
ru = None
if config['use_wandb']:
    run_name = type(model).__name__+str(datetime.now())[8:18]
    wandb.init(
        project=config['project'],
        name = run_name,
        config=config['model'],
    )
train(model=model, optimizer=optimizer, loss_fn=loss_fn, dataloaders=dataloaders,config=config['train'], model_config=config['model'], scheduler = scheduler, device = device, use_wandb = config['use_wandb'])

Validation: 100%|██████████| 20/20 [00:25<00:00,  1.28s/it, val_accuracy=80.5, val_loss=0.61]


In [ ]:
wandb.init(
    project=config['project'],
    name = type(model).__name__+str(datetime.now())[8:18],
    config=config['model'],
)

In [ ]:

model_artifact = wandb.use_artifact("trained-model:latest")
model_dir = model_artifact.download()
model_path = os.path.join(model_dir, "trained_model.pt")
model_config = model_artifact.metadata

model = DefaultModel(**model_config)
model.load_state_dict(torch.load(model_path))
model.to(device)

    loss, accuracy, highest_losses, hardest_examples, true_labels, preds = evaluate(model, test_loader)

    run.summary.update({"loss": loss, "accuracy": accuracy})

    wandb.log({"high-loss-examples":
        [wandb.Image(hard_example, caption=str(int(pred)) + "," +  str(int(label)))
            for hard_example, pred, label in zip(hardest_examples, preds, true_labels)]})


### Load model

In [ ]:
# model_path = './checkpoints/ResNet_cuda2025-04-19 17_1.pth'
# model.load_state_dict(torch.load(model_path, map_location=device))

### Prediction

In [ ]:
from train import visualize_model
visualize_model(model,valdl, class_names, 100, device=device)